<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/indicator/nse_pe_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss

# Suppress warnings (optional)
import warnings
warnings.filterwarnings('ignore')

# Set plot style
sns.set_style('whitegrid')

# Step 1: Data Import and Cleaning
# Assume the CSV file is named 'nifty_pe_data.csv' with columns 'Date' and 'PE_Ratio'

# Read the data
df = pd.read_csv("https://raw.githubusercontent.com/anirbanghoshsbi/.github.io/master/work/ml/data_fundamental.csv")
#df.sort_values('Date', inplace=True)
df.set_index('Date', inplace=True)


# Handle missing values (if any)
#df['feat_PE'].fillna(method='ffill', inplace=True)  # Forward fill
#df['PE_Ratio'].fillna(method='bfill', inplace=True)  # Backward fill if needed

# Step 2: Stationarity Testing
# Define ADF test function
def adf_test(series, title=''):
    print(f'\nAugmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    print(out.to_string())
    if result[1] <= 0.05:
        print("=> The series is stationary.")
    else:
        print("=> The series is non-stationary.")

# Define KPSS test function
def kpss_test(series, title=''):
    print(f'\nKPSS Test: {title}')
    result = kpss(series.dropna(), regression='c', nlags='auto')
    labels = ['KPSS Statistic', 'p-value', '# Lags Used']
    out = pd.Series(result[0:3], index=labels)
    for key, value in result[3].items():
        out[f'Critical Value ({key})'] = value
    print(out.to_string())
    if result[1] <= 0.05:
        print("=> The series is non-stationary.")
    else:
        print("=> The series is stationary.")

# Apply the tests on the original series
adf_test(df['feat_PE'], title='P/E Ratio Original Series')
kpss_test(df['feat_PE'], title='P/E Ratio Original Series')

# Step 3: Differencing to Achieve Stationarity
df['PE_Ratio_Diff'] = df['feat_PE'].diff()

# Apply the tests on the differenced series
adf_test(df['PE_Ratio_Diff'].dropna(), title='First Differenced Series')
kpss_test(df['PE_Ratio_Diff'].dropna(), title='First Differenced Series')
df=df.tail(750)
# Step 4: Rolling Analysis with a 3-Year Window (Assuming Monthly Data)
window_size = 24  # 3 years * 12 months

# Calculate rolling mean and standard deviation
df['Rolling_Mean'] = df['feat_PE'].rolling(window=window_size).mean()
df['Rolling_STD'] = df['feat_PE'].rolling(window=window_size).std()

# Compute rolling Z-scores
df['Rolling_Z_Score'] = (df['feat_PE'] - df['Rolling_Mean']) / df['Rolling_STD']

# Step 5: Setting Overvaluation and Undervaluation Thresholds
overvaluation_threshold = 2
undervaluation_threshold = -2

# Generate signals
df['Signal'] = 0
df.loc[df['Rolling_Z_Score'] > overvaluation_threshold, 'Signal'] = -1  # Overvalued
df.loc[df['Rolling_Z_Score'] < undervaluation_threshold, 'Signal'] = 1   # Undervalued




Augmented Dickey-Fuller Test: P/E Ratio Original Series
ADF Statistic             -3.411548
p-value                    0.010571
# Lags Used               17.000000
# Observations Used     6429.000000
Critical Value (1%)       -3.431368
Critical Value (5%)       -2.861990
Critical Value (10%)      -2.567009
=> The series is stationary.

KPSS Test: P/E Ratio Original Series
KPSS Statistic            5.447347
p-value                   0.010000
# Lags Used              51.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
=> The series is non-stationary.

Augmented Dickey-Fuller Test: First Differenced Series
ADF Statistic          -1.749673e+01
p-value                 4.407364e-30
# Lags Used             1.600000e+01
# Observations Used     6.429000e+03
Critical Value (1%)    -3.431368e+00
Critical Value (5%)    -2.861990e+00
Critical Value (10%)   -2.567009e+00
=> The series is stationary.

K

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import timedelta

# Ensure overvaluation_threshold is a scalar
overvaluation_threshold = 2  # Set your desired threshold value

# Create a copy of the DataFrame without NaN values in the plotting columns
df_clean = df.dropna(subset=['feat_PE', 'Rolling_Mean', 'Rolling_STD', 'Signal', 'Close'])

# Ensure the index is a DatetimeIndex
df_clean.index = pd.to_datetime(df_clean.index)

# Visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Plot P/E Ratio and Rolling Mean with Thresholds
ax1.plot(df_clean.index, df_clean['feat_PE'], label='P/E Ratio', color='blue')
ax1.plot(df_clean.index, df_clean['Rolling_Mean'], label='Rolling Mean (36 days)', color='orange')
ax1.fill_between(
    df_clean.index,
    df_clean['Rolling_Mean'] + overvaluation_threshold * df_clean['Rolling_STD'],
    df_clean['Rolling_Mean'] - overvaluation_threshold * df_clean['Rolling_STD'],
    color='gray', alpha=0.2, label='Threshold Band'
)

# Highlight overvalued and undervalued signals
ax1.scatter(
    df_clean.index[df_clean['Signal'] == -1],
    df_clean['feat_PE'][df_clean['Signal'] == -1],
    marker='v', color='red', label='Overvalued'
)
ax1.scatter(
    df_clean.index[df_clean['Signal'] == 1],
    df_clean['feat_PE'][df_clean['Signal'] == 1],
    marker='^', color='green', label='Undervalued'
)

ax1.set_title('Nifty Index P/E Ratio with Rolling Mean and Thresholds')
ax1.set_ylabel('P/E Ratio')
ax1.legend(loc=2)

# Plot Nifty Close
ax2.plot(df_clean.index, df_clean['Close'], label='Nifty Close', color='purple')
ax2.set_xlabel('Date')
ax2.set_ylabel('Close Price')
ax2.legend(loc='upper left')

# Format x-axis dates
for ax in [ax1, ax2]:
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

fig.autofmt_xdate(rotation=45)

# Extend the x-axis on the right by 5 days
right_limit = df_clean.index.max() + timedelta(days=10)
ax1.set_xlim(left=df_clean.index.min(), right=right_limit)
ax2.set_xlim(left=df_clean.index.min(), right=right_limit)

# Adjust layout to prevent clipping
plt.tight_layout()
plt.subplots_adjust(right=0.95)

plt.show()
